In [1]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

In [3]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": 48.4192,
	"longitude": 9.9491,
	"start_date": "2024-01-01",
	"end_date": "2025-01-01",
	"minutely_15": ["temperature_2m", "rain", "snowfall", "shortwave_radiation", "precipitation", "relative_humidity_2m"],
	"timezone": "UTC"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process minutely_15 data. The order of variables needs to be the same as requested.
minutely_15 = response.Minutely15()
minutely_15_temperature_2m = minutely_15.Variables(0).ValuesAsNumpy()
minutely_15_rain = minutely_15.Variables(1).ValuesAsNumpy()
minutely_15_snowfall = minutely_15.Variables(2).ValuesAsNumpy()
minutely_15_shortwave_radiation = minutely_15.Variables(3).ValuesAsNumpy()
minutely_15_precipitation = minutely_15.Variables(4).ValuesAsNumpy()
minutely_15_relative_humidity_2m = minutely_15.Variables(5).ValuesAsNumpy()

minutely_15_data = {"date": pd.date_range(
	start = pd.to_datetime(minutely_15.Time(), unit = "s", utc = True),
	end = pd.to_datetime(minutely_15.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = minutely_15.Interval()),
	inclusive = "left"
)}

minutely_15_data["temperature_2m"] = minutely_15_temperature_2m
minutely_15_data["rain"] = minutely_15_rain
minutely_15_data["snowfall"] = minutely_15_snowfall
minutely_15_data["shortwave_radiation"] = minutely_15_shortwave_radiation
minutely_15_data["precipitation"] = minutely_15_precipitation
minutely_15_data["relative_humidity_2m"] = minutely_15_relative_humidity_2m

weather_df = pd.DataFrame(data = minutely_15_data)

Coordinates 48.41999816894531°N 9.940000534057617°E
Elevation 615.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s


In [4]:
# 1. Convert the datetime column to a string without timezone info
weather_df['date_clean'] = weather_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

# 2. Parse into datetime (this re-parsing is optional since 'date' is already datetime, but done for clarity)
weather_df['datetime'] = pd.to_datetime(weather_df['date_clean'], format='%Y-%m-%d %H:%M:%S')

# 3. Set datetime as index, drop old columns
weather_df.set_index('datetime', inplace=True)
weather_df = weather_df.drop(columns=['date', 'date_clean'])

In [5]:
weather_df = weather_df.loc['2024']
weather_df

,temperature_2m,rain,snowfall,shortwave_radiation,precipitation,relative_humidity_2m
datetime,,,,,,
2024-01-01 00:00:00,2.25,0.0,0.0,0.0,0.0,91.0
2024-01-01 00:15:00,2.15,0.0,0.0,0.0,0.0,91.0
2024-01-01 00:30:00,2.00,0.0,0.0,0.0,0.0,92.0
2024-01-01 00:45:00,1.85,0.0,0.0,0.0,0.0,93.0
2024-01-01 01:00:00,1.75,0.0,0.0,0.0,0.0,93.0
...,...,...,...,...,...,...
2024-12-31 22:45:00,-1.60,0.0,0.0,0.0,0.0,86.0
2024-12-31 23:00:00,-1.60,0.0,0.0,0.0,0.0,86.0
2024-12-31 23:15:00,-1.60,0.0,0.0,0.0,0.0,86.0


In [ ]:
weather_df.to_csv('weather.csv', index_label='Datetime')